## Torch.squeeze( input, dim) 

* Tensor의 dimension을 맞추기 위해서 dimension을 변경해야 할 경우가 있습니다. 특히 이번에 알아볼 경우는 dimension을 축소하는 경우입니다. 


* dimension 축소를 위해서는 tensor.squeeze() 함수를 이용하여 아래와 같이 응용할 수 있습니다. 


* torch.squeeze() 함수는 어떤 dimension 값이 1일 때, 그 dimension을 제거해줍니다.

In [7]:
import torch

tensor = torch.rand(1,5,5)

print(tensor.shape)

tensor = torch.squeeze(tensor, 0)
print(tensor.shape)

tensor = torch.rand(1, 5, 5)
tensor = torch.squeeze(tensor, 1)

print(tensor.shape)

torch.Size([1, 5, 5])
torch.Size([5, 5])
torch.Size([1, 5, 5])


* 위 예제와 같이 torch.squeeze(tensor, 1) 에서는 dimension의 값이 1이 아니므로 dimension이 제거되지 않았습니다.

# blcok을 쌓기 위한 Module, Sequential, ModuleList, ModuleDict

* 출처 :https://towardsdatascience.com/pytorch-how-and-when-to-use-module-sequential-modulelist-and-moduledict-7a54597b5f17


* torch.nn에 있는 Module, Sequential, ModuleList, ModuleDict는 모두 Network block을 쌓기 위하여 사용되는 클래스입니다. 즉, 다음과 같이 사용할 수 있습니다.

In [8]:
import torch.nn as nn

* 먼저 각 기능들을 언제 사용할 지 정리해 본 후 차례대로 설명하겠습니다.


* Module : 여러 개의 작은 블록으로 구성된 큰 블록이 있을 때

* Sequential : 레이어에서 작은 블록을 만들고 싶을 때 

* ModuleList : 일부 레이어 또는 빌딩 블록을 반복하면서 어떤 작업을 해야할 때

* ModuleDict : 모델의 일부 블록을 매개 변수화 해야하는 경우 ( 예: activation 기능 )

# Module : The Main building block

* 먼저 Module 에 대해서 알아보도록 하겠습니다. Module은 가장 기본이 되는 block 단위입니다. 

* 따라서 모든 pytorch의 기본 block들은 Module에서부터 상속 받아서 사용되므로 Network를 만들 때 반드시 사용됩니다. 

* 그러면 Sequential 또는 ModuleList 없이 단순히 Module만 사용한 아래 예제를 살펴보도록 하겠습니다. 



In [9]:
import torch.nn.functional as F 

class CNNclassifier(nn.Module):
    def __inti__(self, in_c, n_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(in_c, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        
        self.fc1 = nn.Linear(32 * 28 * 28, 1024)
        self.fc2 = nn.Linear(1024, n_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)

        x = x.view(x.size(0), -1) # flat
        
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        
        return x

* 상당히 단순한 모델입니다. forward를 보면 Convolution → BatchNorm → ReLu로 이어지는 블록을 차례대로 이은 구조입니다.


* __init__에서 선언된 각 객체들이 Module block 입니다. (nn.Conv2d, nn.BatchNor2d 등)


* 위 코드를 보면 Convolution → BatchNorm → ReLu 블록이 이어져서 사용됨에도 불구하고 함수 처럼 사용하지 못하는 것은 다소 비효율적으로 보입니다. 이것을 개선하기 위하여 Sequential과 ModuleList를 사용할 수 있습니다.

## Sequential: stack and merge layers


* 그 다음으로는 Sequential 입니다. Sequential은 마치 컨태이너 처럼 Module을 담는 역할을 합니다. Sequential에 쌓은 순서대로 Module은 실행되고 같은 Sequential에 쌓인 Module 들은 한 단위처럼 실행됩니다.


* 따라서 Module 중에서 동시에 쓰이는 것을 Sequential로 묶어서 사용하면 코드가 간단해집니다.


* 예를 들어 위 예제에서 Convolution → BatchNorm → ReLu는 3개의 Module이 연달아 사용되기 때문에 마치 하나의 단위처럼 생각할 수 있습니다. 따라서 이 3개의 Module을 Sequential로 만들어 보겠습니다.

In [10]:
class CNNClassifier(nn.Module):
    def __init__(self, in_c, n_classes):
        super().__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_c, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(32 * 28 * 28, 1024),
            nn.Sigmoid(),
            nn.Linear(1024, n_classes)
        )

        
    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)

        x = x.view(x.size(0), -1) # flat
        
        x = self.decoder(x)
        
        return x

* 위 코드를 보면 __init__에서도 단위 별로 묶어서 표현할 수 있고 forward에서는 코드가 훨씬 간결해 진것을 확인할 수 있습니다.


* 위 코드에서 conv_block1과 conv_block2 또한 코드가 중복되었습니다. 중복되는 코드를 함수로 빼면 더 간결하게 쓸 수 있습니다.

In [11]:
def conv_block(in_f, out_f, *args, **kwargs):
    return nn.Sequential(
        nn.Conv2d(in_f, out_f, *args, **kwargs),
        nn.BatchNorm2d(out_f),
        nn.ReLU()
    )

class CNNClassifier(nn.Module):
    def __init__(self, in_c, n_classes):
        super().__init__()
        self.conv_block1 = conv_block(in_c, 32, kernel_size=3, padding=1)
        
        self.conv_block2 = conv_block(32, 64, kernel_size=3, padding=1)

        
        self.decoder = nn.Sequential(
            nn.Linear(32 * 28 * 28, 1024),
            nn.Sigmoid(),
            nn.Linear(1024, n_classes)
        )

        
    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)

        x = x.view(x.size(0), -1) # flat
        
        x = self.decoder(x)
        
        return x

* 위 코드도 충분히 깔끔하지만 더 큰 네트워크를 쌓기 위해서 위 코드를 더 깔끔하게 만들어 보겠습니다. 다음부터 쓰이는 기법들은 큰 네트워크를 쌓을 때 상당히 도움이 됩니다.

In [12]:
def conv_block(in_f, out_f, *args, **kwargs):
    return nn.Sequential(
        nn.Conv2d(in_f, out_f, *args, **kwargs),
        nn.BatchNorm2d(out_f),
        nn.ReLU()
    )

class CNNClassifier(nn.Module):
    def __init__(self, in_c, n_classes):
        super().__init__()
        self.encoder = nn.Sequential(
            conv_block(in_c, 32, kernel_size=3, padding=1),
            conv_block(32, 64, kernel_size=3, padding=1)
        )

        self.decoder = nn.Sequential(
            nn.Linear(32 * 28 * 28, 1024),
            nn.Sigmoid(),
            nn.Linear(1024, n_classes)
        )

        
    def forward(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1) # flat
        x = self.decoder(x)
        return x

* 만약 위 코드에서 사용된 self.encoder 부분이 계속 늘어난다면 단순히 코드를 나열하는 것은 좋은 방법이 아닙니다. 예를 들어 다음 예는 별로 좋지 않습니다.

In [ ]:
self.encoder = nn.Sequential(
            conv_block(in_c, 32, kernel_size=3, padding=1),
            conv_block(32, 64, kernel_size=3, padding=1),
            conv_block(64, 128, kernel_size=3, padding=1),
            conv_block(128, 256, kernel_size=3, padding=1),

        )

* 이런 경우 당연히 반복문을 이용하여 코드를 간결하게 작성할 수 있습니다. 이 때 반복문을 진행하면서 변경해주어야 할 것은 input과 output의 channel 수 입니다.


* input과 output의 channel 수는 list를 이용하여 정의해 두는 방법을 많이 사용합니다. 간단하기 때문입니다. 핵심은 반복문을 사용하되 channel의 크기는 미리 저장해 두고 사용하면 된다는 것입니다. 

In [18]:
class CNNClassifier(nn.Module):
    def __init__(self, in_c, n_classes):
        super().__init__()
        self.enc_sizes = [in_c, 32, 64]
        conv_blocks = [conv_block(in_f, out_f, kernel_size=3, padding=1) 
                       for in_f, out_f in zip(self.enc_sizes, self.enc_sizes[1:])]
        self.encoder = nn.Sequential(*conv_blocks)
        
        self.decoder = nn.Sequential(
            nn.Linear(32 * 28 * 28, 1024),
            nn.Sigmoid(),
            nn.Linear(1024, n_classes)
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1) # flat
        x = self.decoder(x)
        
        return x


* 위 코드를 보면 conv_blocks에서 블록을 convolution 블록을 생성합니다. 이 때 self.enc_sizes 리스트를 이용하여 input channel과 output의 channel을 정의해 줍니다. 당연히 n번째 block의 output channel 수가 n+1 번째 block의 input channel 수가 되므로 이를 이용하여 리스트를 교차해서 접근하면 됩니다. (위 코드에서도 이 방법을 사용하였습니다.)


* 연산자를 리스트와 같이 사용하면 아래와 같이 편하게 사용할 수 있습니다. 상세내용은 다음 링크를 참조하시기 바랍니다. -링크 : https://gaussian37.github.io/python-basic-asterisk/

In [19]:
a = [1, 2, 3, 4, 5]
b = [10, *a]
print(b)

[10, 1, 2, 3, 4, 5]
